ЧАСТЬ 1: ЗАГРУЗКА И ПОДГОТОВКА ДАННЫХ
Загружаем исходные данные и преобразуем их в удобный для анализа формат. Нормализуем названия месяцев для единообразия.

In [64]:
# === ЧАСТЬ 1: ЗАГРУЗКА И ПОДГОТОВКА ДАННЫХ ===

import pandas as pd
import numpy as np

print("Загрузка данных...")

# Загрузка данных
prolongations = pd.read_csv('prolongations.csv')
financial_data = pd.read_csv('financial_data.csv')

print(f"prolongations: {len(prolongations)} записей")
print(f"financial_data: {len(financial_data)} записей")

# Создаем mapping для месяцев
month_mapping = {
    'ноябрь 2022': '2022-11', 'декабрь 2022': '2022-12',
    'январь 2023': '2023-01', 'февраль 2023': '2023-02', 'март 2023': '2023-03',
    'апрель 2023': '2023-04', 'май 2023': '2023-05', 'июнь 2023': '2023-06',
    'июль 2023': '2023-07', 'август 2023': '2023-08', 'сентябрь 2023': '2023-09',
    'октябрь 2023': '2023-10', 'ноябрь 2023': '2023-11', 'декабрь 2023': '2023-12'
}

# Нормализуем месяцы в prolongations
prolongations['month_norm'] = prolongations['month'].str.lower().map(month_mapping)

# Преобразуем financial_data из широкого в длинный формат
month_columns = [col for col in financial_data.columns if col not in ['id', 'Причина дубля', 'Account']]
financial_data_long = financial_data.melt(
    id_vars=['id', 'Причина дубля', 'Account'],
    value_vars=month_columns,
    var_name='month',
    value_name='shipment_value'
)

# Нормализуем месяцы в financial_data
financial_data_long['month_norm'] = financial_data_long['month'].str.lower().map(month_mapping)

print("Данные подготовлены")

Загрузка данных...
prolongations: 477 записей
financial_data: 451 записей
Данные подготовлены


ЧАСТЬ 2: ОБРАБОТКА ФИНАНСОВЫХ ДАННЫХ
Преобразуем финансовые данные из строкового формата в числовой. Обрабатываем специальные значения согласно требованиям ТЗ: 'стоп', 'end', 'в ноль'.

In [65]:
# === ЧАСТЬ 2: ОБРАБОТКА ФИНАНСОВЫХ ДАННЫХ ===

print("Обработка финансовых данных...")

def parse_shipment_value(value):
    """
    Преобразует значение отгрузки в числовой формат
    Обрабатывает специальные значения: 'стоп', 'end', 'в ноль'
    """
    if pd.isna(value):
        return 0, 'empty'

    value_str = str(value).strip()

    if value_str.lower() in ['стоп', 'end']:
        return 0, 'stopped'
    elif value_str.lower() == 'в ноль':
        return 0, 'zero'
    else:
        try:
            # Убираем пробелы (разделители тысяч) и заменяем запятые на точки
            cleaned_value = value_str.replace(' ', '').replace(',', '.')
            numeric_value = float(cleaned_value)
            return numeric_value, 'normal'
        except:
            # Пробуем извлечь только цифры, точки и запятые
            import re
            numbers_only = re.findall(r'[\d,\.]+', value_str)
            if numbers_only:
                cleaned_value = numbers_only[0].replace(',', '.')
                numeric_value = float(cleaned_value)
                return numeric_value, 'normal'
            else:
                return 0, 'invalid'

# Применяем функцию к данным
shipment_parsed = financial_data_long['shipment_value'].apply(parse_shipment_value)
financial_data_long['shipment_amount'] = [x[0] for x in shipment_parsed]
financial_data_long['shipment_status'] = [x[1] for x in shipment_parsed]

print("Финансовые данные обработаны")
print(f"Статусы отгрузок:")
print(financial_data_long['shipment_status'].value_counts())

Обработка финансовых данных...
Финансовые данные обработаны
Статусы отгрузок:
shipment_status
empty      4620
normal     2394
zero        129
stopped      73
Name: count, dtype: int64


ЧАСТЬ 3: РАСЧЕТ КОЭФФИЦИЕНТОВ ПРОЛОНГАЦИИ
Основная функция расчета коэффициентов по методологии ТЗ. Рассчитываем коэффициенты для каждого месяца 2023 года и агрегируем результаты по менеджерам и отделу.

In [66]:
# === ЧАСТЬ 3: РАСЧЕТ КОЭФФИЦИЕНТОВ ПРОЛОНГАЦИИ ===

print("Расчет коэффициентов пролонгации...")

def calculate_prolongation_coefficients(prolongations_df, financial_df):
    """
    Расчет коэффициентов пролонгации для каждого менеджера и для всего отдела
    """

    # Месяцы для анализа (2023 год с марта по декабрь)
    analysis_months = [f'2023-{str(i).zfill(2)}' for i in range(3, 13)]

    manager_results = []
    department_results = []

    for current_month in analysis_months:
        print(f"Анализ месяца: {current_month}")

        # Определение предыдущих месяцев
        year = int(current_month[:4])
        month_num = int(current_month[5:7])

        # Предыдущий месяц (M-1)
        if month_num > 1:
            prev_month_1 = f"{year}-{str(month_num-1).zfill(2)}"
        else:
            prev_month_1 = f"{year-1}-12"

        # Месяц два назад (M-2)
        if month_num > 2:
            prev_month_2 = f"{year}-{str(month_num-2).zfill(2)}"
        elif month_num == 2:
            prev_month_2 = f"{year-1}-12"
        else:
            prev_month_2 = f"{year-1}-11"

        print(f"  M-1: {prev_month_1}, M-2: {prev_month_2}")

        # 1. КОЭФФИЦИЕНТ 1: Пролонгация в первый месяц

        # Проекты, завершившиеся в M-1
        projects_ended_m1 = prolongations_df[prolongations_df['month_norm'] == prev_month_1]
        print(f"  Проектов завершилось в {prev_month_1}: {len(projects_ended_m1)}")

        coeff_1_data = []

        for _, project in projects_ended_m1.iterrows():
            project_id = project['id']
            manager = project['AM']

            # Получаем данные проекта в M-1
            project_data_m1 = financial_df[
                (financial_df['id'] == project_id) &
                (financial_df['month_norm'] == prev_month_1)
            ]

            # Исключаем проекты со статусом 'stopped'
            if len(project_data_m1) > 0 and any(project_data_m1['shipment_status'] == 'stopped'):
                continue

            # Сумма отгрузок в M-1 (последний месяц реализации)
            shipment_m1 = project_data_m1['shipment_amount'].sum()

            # Отгрузка в текущем месяце M
            shipment_current = financial_df[
                (financial_df['id'] == project_id) &
                (financial_df['month_norm'] == current_month)
            ]['shipment_amount'].sum()

            if shipment_m1 > 0:
                coeff_1_data.append({
                    'manager': manager,
                    'shipment_prev': shipment_m1,
                    'shipment_current': shipment_current
                })

        # Расчет коэффициента 1
        if coeff_1_data:
            coeff_1_df = pd.DataFrame(coeff_1_data)
            manager_coeff_1 = coeff_1_df.groupby('manager').agg({
                'shipment_prev': 'sum',
                'shipment_current': 'sum'
            }).reset_index()

            manager_coeff_1['coeff_1_month'] = manager_coeff_1['shipment_current'] / manager_coeff_1['shipment_prev']
            manager_coeff_1['month'] = current_month

            # Общий коэффициент по отделу
            total_prev_1 = coeff_1_df['shipment_prev'].sum()
            total_current_1 = coeff_1_df['shipment_current'].sum()
            total_coeff_1 = total_current_1 / total_prev_1 if total_prev_1 > 0 else 0

            print(f"  Коэффициент 1: {total_coeff_1:.3f} ({len(coeff_1_data)} проектов)")
        else:
            manager_coeff_1 = pd.DataFrame(columns=['manager', 'coeff_1_month', 'month'])
            total_coeff_1 = 0
            print(f"  Коэффициент 1: нет данных")

        # 2. КОЭФФИЦИЕНТ 2: Пролонгация во второй месяц

        # Проекты, завершившиеся в M-2
        projects_ended_m2 = prolongations_df[prolongations_df['month_norm'] == prev_month_2]
        print(f"  Проектов завершилось в {prev_month_2}: {len(projects_ended_m2)}")

        coeff_2_data = []

        for _, project in projects_ended_m2.iterrows():
            project_id = project['id']
            manager = project['AM']

            # Проверяем, что проект НЕ был пролонгирован в M-1
            shipment_m1_check = financial_df[
                (financial_df['id'] == project_id) &
                (financial_df['month_norm'] == prev_month_1)
            ]['shipment_amount'].sum()

            if shipment_m1_check > 0:
                continue

            # Получаем данные проекта в M-2
            project_data_m2 = financial_df[
                (financial_df['id'] == project_id) &
                (financial_df['month_norm'] == prev_month_2)
            ]

            # Исключаем проекты со статусом 'stopped'
            if len(project_data_m2) > 0 and any(project_data_m2['shipment_status'] == 'stopped'):
                continue

            # Сумма отгрузок в M-2
            shipment_m2 = project_data_m2['shipment_amount'].sum()

            # Отгрузка в текущем месяце M
            shipment_current_2 = financial_df[
                (financial_df['id'] == project_id) &
                (financial_df['month_norm'] == current_month)
            ]['shipment_amount'].sum()

            if shipment_m2 > 0:
                coeff_2_data.append({
                    'manager': manager,
                    'shipment_prev': shipment_m2,
                    'shipment_current': shipment_current_2
                })

        # Расчет коэффициента 2
        if coeff_2_data:
            coeff_2_df = pd.DataFrame(coeff_2_data)
            manager_coeff_2 = coeff_2_df.groupby('manager').agg({
                'shipment_prev': 'sum',
                'shipment_current': 'sum'
            }).reset_index()

            manager_coeff_2['coeff_2_month'] = manager_coeff_2['shipment_current'] / manager_coeff_2['shipment_prev']
            manager_coeff_2['month'] = current_month

            # Общий коэффициент по отделу
            total_prev_2 = coeff_2_df['shipment_prev'].sum()
            total_current_2 = coeff_2_df['shipment_current'].sum()
            total_coeff_2 = total_current_2 / total_prev_2 if total_prev_2 > 0 else 0

            print(f"  Коэффициент 2: {total_coeff_2:.3f} ({len(coeff_2_data)} проектов)")
        else:
            manager_coeff_2 = pd.DataFrame(columns=['manager', 'coeff_2_month', 'month'])
            total_coeff_2 = 0
            print(f"  Коэффициент 2: нет данных")

        # Сохраняем результаты
        if len(manager_coeff_1) > 0:
            manager_month_result = manager_coeff_1[['manager', 'month', 'coeff_1_month']].rename(columns={'manager': 'AM'})

            if len(manager_coeff_2) > 0:
                manager_coeff_2_clean = manager_coeff_2[['manager', 'coeff_2_month']].rename(columns={'manager': 'AM'})
                manager_month_result = manager_month_result.merge(manager_coeff_2_clean, on='AM', how='left')
            else:
                manager_month_result['coeff_2_month'] = 0

            manager_results.append(manager_month_result)

        # Результаты по отделу
        dept_result = {
            'month': current_month,
            'total_coeff_1_month': total_coeff_1,
            'total_coeff_2_month': total_coeff_2
        }
        department_results.append(dept_result)

        print("")

    # Объединение результатов
    final_manager_results = pd.concat(manager_results, ignore_index=True) if manager_results else pd.DataFrame()
    final_department_results = pd.DataFrame(department_results)

    return final_manager_results, final_department_results

# Запуск расчета
print("Начало расчета коэффициентов пролонгации...")
manager_coefficients, department_coefficients = calculate_prolongation_coefficients(prolongations, financial_data_long)

print("Расчет коэффициентов завершен")
print(f"Результаты для менеджеров: {len(manager_coefficients)} записей")
print(f"Результаты для отдела: {len(department_coefficients)} месяцев")

Расчет коэффициентов пролонгации...
Начало расчета коэффициентов пролонгации...
Анализ месяца: 2023-03
  M-1: 2023-02, M-2: 2023-01
  Проектов завершилось в 2023-02: 28
  Коэффициент 1: 0.646 (20 проектов)
  Проектов завершилось в 2023-01: 20
  Коэффициент 2: 0.162 (10 проектов)

Анализ месяца: 2023-04
  M-1: 2023-03, M-2: 2023-02
  Проектов завершилось в 2023-03: 33
  Коэффициент 1: 0.445 (32 проектов)
  Проектов завершилось в 2023-02: 28
  Коэффициент 2: 0.091 (8 проектов)

Анализ месяца: 2023-05
  M-1: 2023-04, M-2: 2023-03
  Проектов завершилось в 2023-04: 28
  Коэффициент 1: 0.568 (23 проектов)
  Проектов завершилось в 2023-03: 33
  Коэффициент 2: 0.000 (16 проектов)

Анализ месяца: 2023-06
  M-1: 2023-05, M-2: 2023-04
  Проектов завершилось в 2023-05: 22
  Коэффициент 1: 0.222 (19 проектов)
  Проектов завершилось в 2023-04: 28
  Коэффициент 2: 0.038 (6 проектов)

Анализ месяца: 2023-07
  M-1: 2023-06, M-2: 2023-05
  Проектов завершилось в 2023-06: 31
  Коэффициент 1: 0.529 (27 пр

ЧАСТЬ 4: АГРЕГАЦИЯ РЕЗУЛЬТАТОВ
Суммируем результаты расчетов за год. Определяем средние значения коэффициентов, строим рейтинги менеджеров по эффективности пролонгаций.

In [67]:
# === ЧАСТЬ 4: АГРЕГАЦИЯ РЕЗУЛЬТАТОВ ===

print("Агрегация результатов...")

# Годовые коэффициенты для отдела
yearly_dept_coeff_1 = department_coefficients['total_coeff_1_month'].mean()
yearly_dept_coeff_2 = department_coefficients['total_coeff_2_month'].mean()

print(f"Средние коэффициенты за 2023 год:")
print(f"Коэффициент 1 (пролонгация в первый месяц): {yearly_dept_coeff_1:.3f}")
print(f"Коэффициент 2 (пролонгация во второй месяц): {yearly_dept_coeff_2:.3f}")

# Годовые коэффициенты для менеджеров
yearly_manager_results = manager_coefficients.groupby('AM').agg({
    'coeff_1_month': ['mean', 'count'],
    'coeff_2_month': ['mean', 'count']
}).round(3)

yearly_manager_results.columns = ['coeff_1_avg', 'coeff_1_months', 'coeff_2_avg', 'coeff_2_months']
yearly_manager_results = yearly_manager_results.reset_index()

print(f"Проанализировано менеджеров: {len(yearly_manager_results)}")
print(f"Общее количество расчетных периодов: {len(manager_coefficients)}")

# Рейтинги менеджеров
coeff_1_ranking = yearly_manager_results[['AM', 'coeff_1_avg']].sort_values('coeff_1_avg', ascending=False)
coeff_2_ranking = yearly_manager_results[['AM', 'coeff_2_avg']].sort_values('coeff_2_avg', ascending=False)

print("\nТоп-3 менеджера по коэффициенту 1:")
for i, (_, row) in enumerate(coeff_1_ranking.head(3).iterrows(), 1):
    print(f"  {i}. {row['AM']}: {row['coeff_1_avg']:.3f}")

Агрегация результатов...
Средние коэффициенты за 2023 год:
Коэффициент 1 (пролонгация в первый месяц): 0.512
Коэффициент 2 (пролонгация во второй месяц): 0.068
Проанализировано менеджеров: 8
Общее количество расчетных периодов: 57

Топ-3 менеджера по коэффициенту 1:
  1. Петрова Анна Дмитриевна: 1.112
  2. Михайлов Андрей Сергеевич: 0.792
  3. Соколова Анастасия Викторовна: 0.629


ЧАСТЬ 5: СОЗДАНИЕ ОСНОВНОГО ОТЧЕТА
Формируем детализированный Excel отчет с полной информацией для углубленного анализа. Отчет содержит несколько листов с разной степенью детализации.

In [68]:
# === ЧАСТЬ 5: СОЗДАНИЕ ОТЧЕТОВ ===

print("Создание отчетов...")

# Основной отчет с детализацией
output_file = "Отчет_по_пролонгациям_2023.xlsx"

with pd.ExcelWriter(output_file, engine='openpyxl') as writer:

    # 1. Сводный лист с основными показателями
    summary_data = {
        'Показатель': [
            'Средний коэффициент пролонгации в первый месяц (отдел)',
            'Средний коэффициент пролонгации во второй месяц (отдел)',
            'Период анализа',
            'Количество анализируемых проектов',
            'Количество менеджеров в анализе',
            'Лучший менеджер по коэффициенту 1',
            'Лучший месяц по эффективности'
        ],
        'Значение': [
            f"{yearly_dept_coeff_1:.3f}",
            f"{yearly_dept_coeff_2:.3f}",
            "Март-Декабрь 2023",
            f"{prolongations['id'].nunique()}",
            f"{len(yearly_manager_results)}",
            f"{coeff_1_ranking.iloc[0]['AM']}",
            f"{department_coefficients.loc[department_coefficients['total_coeff_1_month'].idxmax(), 'month']}"
        ]
    }

    summary_df = pd.DataFrame(summary_data)
    summary_df.to_excel(writer, sheet_name='Сводка', index=False)

    # 2. Годовые результаты по менеджерам
    yearly_manager_results.to_excel(writer, sheet_name='Менеджеры_годовые', index=False)

    # 3. Месячные результаты по менеджерам
    manager_coefficients.to_excel(writer, sheet_name='Менеджеры_месячные', index=False)

    # 4. Результаты по отделу
    department_coefficients_clean = department_coefficients.copy()
    department_coefficients_clean['total_coeff_1_month'] = department_coefficients_clean['total_coeff_1_month'].round(3)
    department_coefficients_clean['total_coeff_2_month'] = department_coefficients_clean['total_coeff_2_month'].round(3)
    department_coefficients_clean.to_excel(writer, sheet_name='Отдел_месячные', index=False)

    # 5. Рейтинги менеджеров
    coeff_1_ranking.to_excel(writer, sheet_name='Рейтинг_К1', index=False)
    coeff_2_ranking.to_excel(writer, sheet_name='Рейтинг_К2', index=False)

print(f"Основной отчет создан: {output_file}")

Создание отчетов...
Основной отчет создан: Отчет_по_пролонгациям_2023.xlsx


ЧАСТЬ 6: СОЗДАНИЕ УПРОЩЕННОГО ОТЧЕТА
Создаем версию отчета для руководителя с ключевыми метриками и выводами. Отчет ориентирован на быстрое принятие управленческих решений.

In [69]:
# === ЧАСТЬ 6: СОЗДАНИЕ УПРОЩЕННОГО ОТЧЕТА ДЛЯ РУКОВОДИТЕЛЯ ===

print("Создание упрощенного отчета для руководителя...")

simple_output_file = "Отчет_по_пролонгациям_упрощенный.xlsx"

with pd.ExcelWriter(simple_output_file, engine='openpyxl') as writer:

    # Основные показатели для руководителя
    simple_data = {
        'Метрика': [
            'Общая эффективность пролонгаций',
            'Пролонгация в первый месяц (К1)',
            'Пролонгация во второй месяц (К2)',
            'Лучший менеджер по К1',
            'Лучший месяц по эффективности',
            'Ключевая рекомендация'
        ],
        'Значение': [
            f"{(yearly_dept_coeff_1 + yearly_dept_coeff_2) / 2:.1%}",
            f"{yearly_dept_coeff_1:.1%}",
            f"{yearly_dept_coeff_2:.1%}",
            f"{coeff_1_ranking.iloc[0]['AM']} ({coeff_1_ranking.iloc[0]['coeff_1_avg']:.1%})",
            f"{department_coefficients.loc[department_coefficients['total_coeff_1_month'].idxmax(), 'month']}",
            "Улучшить работу с проектами, не пролонгированными в первый месяц"
        ]
    }

    simple_df = pd.DataFrame(simple_data)
    simple_df.to_excel(writer, sheet_name='Основные_показатели', index=False)

    # Топ-5 менеджеров для мотивации
    top_managers = coeff_1_ranking.head().copy()
    top_managers['coeff_1_avg'] = top_managers['coeff_1_avg'].apply(lambda x: f"{x:.1%}")
    top_managers.columns = ['Менеджер', 'Эффективность пролонгаций']
    top_managers.to_excel(writer, sheet_name='Топ_менеджеры', index=False)

    # Динамика по месяцам
    monthly_dynamics = department_coefficients_clean.copy()
    monthly_dynamics['total_coeff_1_month'] = monthly_dynamics['total_coeff_1_month'].apply(lambda x: f"{x:.1%}")
    monthly_dynamics['total_coeff_2_month'] = monthly_dynamics['total_coeff_2_month'].apply(lambda x: f"{x:.1%}")
    monthly_dynamics.columns = ['Месяц', 'Пролонгация в 1-й месяц', 'Пролонгация во 2-й месяц']
    monthly_dynamics.to_excel(writer, sheet_name='Динамика_по_месяцам', index=False)

print(f"Упрощенный отчет создан: {simple_output_file}")

Создание упрощенного отчета для руководителя...
Упрощенный отчет создан: Отчет_по_пролонгациям_упрощенный.xlsx


In [70]:
# === ЧАСТЬ 7: ДОБАВЛЕНИЕ ГРАФИКОВ В EXCEL ОТЧЕТ ===

print("Добавление графиков в Excel отчет...")

from openpyxl import load_workbook
from openpyxl.chart import LineChart, BarChart, Reference
import shutil

# Создаем копию основного отчета с графиками
output_file_with_charts = "Отчет_по_пролонгациям_2023_с_графиками.xlsx"
shutil.copy('Отчет_по_пролонгациям_2023.xlsx', output_file_with_charts)

# Загружаем workbook
workbook = load_workbook(output_file_with_charts)

# Создаем лист для визуализаций
if 'Визуализации' in workbook.sheetnames:
    del workbook['Визуализации']
viz_sheet = workbook.create_sheet('Визуализации')

# Заголовок
viz_sheet['A1'] = 'ВИЗУАЛИЗАЦИИ ЭФФЕКТИВНОСТИ ПРОЛОНГАЦИЙ 2023'
viz_sheet['A1'].font = openpyxl.styles.Font(bold=True, size=14)

# 1. Подготавливаем данные для графика динамики
viz_sheet['A3'] = 'Динамика коэффициентов по месяцам:'
viz_sheet['A3'].font = openpyxl.styles.Font(bold=True)

# Заголовки данных
viz_sheet['A5'] = 'Месяц'
viz_sheet['B5'] = 'Коэффициент 1'
viz_sheet['C5'] = 'Коэффициент 2'

# Данные для графика
for i, (month, coeff1, coeff2) in enumerate(zip(
    department_coefficients['month'],
    department_coefficients['total_coeff_1_month'],
    department_coefficients['total_coeff_2_month']
), 6):
    viz_sheet[f'A{i}'] = month
    viz_sheet[f'B{i}'] = coeff1
    viz_sheet[f'C{i}'] = coeff2

# Создаем линейный график
chart1 = LineChart()
chart1.title = "Динамика коэффициентов пролонгации"
chart1.style = 13
chart1.y_axis.title = "Значение коэффициента"
chart1.x_axis.title = "Месяц 2023 года"
chart1.width = 20
chart1.height = 10

# Данные для графика
data = Reference(viz_sheet, min_col=2, min_row=5, max_row=5+len(department_coefficients), max_col=3)
categories = Reference(viz_sheet, min_col=1, min_row=6, max_row=5+len(department_coefficients))

chart1.add_data(data, titles_from_data=True)
chart1.set_categories(categories)

# Размещаем график
viz_sheet.add_chart(chart1, "E3")

# 2. График эффективности менеджеров
viz_sheet['A25'] = 'Топ-5 менеджеров по эффективности:'
viz_sheet['A25'].font = openpyxl.styles.Font(bold=True)

# Заголовки
viz_sheet['A27'] = 'Менеджер'
viz_sheet['B27'] = 'Коэффициент 1'

# Данные топ-5 менеджеров
top_5_managers = coeff_1_ranking.head()
for i, (manager, coeff1) in enumerate(zip(top_5_managers['AM'], top_5_managers['coeff_1_avg']), 28):
    viz_sheet[f'A{i}'] = manager
    viz_sheet[f'B{i}'] = coeff1

# Создаем столбчатую диаграмму
chart2 = BarChart()
chart2.title = "Топ-5 менеджеров по эффективности"
chart2.style = 10
chart2.y_axis.title = "Средний коэффициент"
chart2.width = 15
chart2.height = 8

data2 = Reference(viz_sheet, min_col=2, min_row=27, max_row=27+len(top_5_managers))
categories2 = Reference(viz_sheet, min_col=1, min_row=28, max_row=27+len(top_5_managers))

chart2.add_data(data2, titles_from_data=True)
chart2.set_categories(categories2)

# Размещаем график
viz_sheet.add_chart(chart2, "E25")

# 3. Добавляем итоговую статистику
viz_sheet['A40'] = 'ИТОГОВАЯ СТАТИСТИКА:'
viz_sheet['A40'].font = openpyxl.styles.Font(bold=True, size=12, color="FF0000")

stats_data = [
    ['Показатель', 'Значение'],
    ['Средний коэффициент 1', f"{yearly_dept_coeff_1:.3f}"],
    ['Средний коэффициент 2', f"{yearly_dept_coeff_2:.3f}"],
    ['Лучший менеджер', f"{coeff_1_ranking.iloc[0]['AM']}"],
    ['Лучший месяц', f"{department_coefficients.loc[department_coefficients['total_coeff_1_month'].idxmax(), 'month']}"],
    ['Худший месяц', f"{department_coefficients.loc[department_coefficients['total_coeff_1_month'].idxmin(), 'month']}"]
]

for row_idx, row_data in enumerate(stats_data, 41):
    for col_idx, value in enumerate(row_data, 1):
        cell = viz_sheet.cell(row=row_idx, column=col_idx, value=value)
        if row_idx == 41:  # Заголовок
            cell.font = openpyxl.styles.Font(bold=True)

# Сохраняем workbook
workbook.save(output_file_with_charts)

print(f"Отчет с графиками создан: {output_file_with_charts}")
print("В отчет добавлены:")
print("- Линейный график динамики коэффициентов")
print("- Столбчатая диаграмма топ-менеджеров")
print("- Итоговая статистика")

Добавление графиков в Excel отчет...
Отчет с графиками создан: Отчет_по_пролонгациям_2023_с_графиками.xlsx
В отчет добавлены:
- Линейный график динамики коэффициентов
- Столбчатая диаграмма топ-менеджеров
- Итоговая статистика


ЧАСТЬ 7: ДОБАВЛЕНИЕ ГРАФИКОВ В EXCEL ОТЧЕТ
Встраиваем интерактивные графики прямо в Excel файл. Создаем линейный график динамики коэффициентов и столбчатую диаграмму рейтинга менеджеров. Графики будут доступны для взаимодействия в Excel.

In [72]:
# === ЧАСТЬ 8: ФИНАЛЬНЫЕ ВЫВОДЫ ===

print("="*60)
print("АНАЛИЗ ЭФФЕКТИВНОСТИ ПРОЛОНГАЦИЙ")
print("="*60)

print(f"\nОСНОВНЫЕ РЕЗУЛЬТАТЫ:")
print(f"• Пролонгация в первый месяц: {yearly_dept_coeff_1:.1%}")
print(f"• Пролонгация во второй месяц: {yearly_dept_coeff_2:.1%}")
print(f"• Лучший менеджер: {coeff_1_ranking.iloc[0]['AM']}")
print(f"• Лучший месяц: {department_coefficients.loc[department_coefficients['total_coeff_1_month'].idxmax(), 'month']}")

print(f"\nСОЗДАННЫЕ ФАЙЛЫ:")
print(f"1. Отчет_по_пролонгациям_2023_с_графиками.xlsx - Полный отчет с графиками")
print(f"2. Отчет_по_пролонгациям_упрощенный.xlsx - Упрощенная версия")


АНАЛИЗ ЭФФЕКТИВНОСТИ ПРОЛОНГАЦИЙ

ОСНОВНЫЕ РЕЗУЛЬТАТЫ:
• Пролонгация в первый месяц: 51.2%
• Пролонгация во второй месяц: 6.8%
• Лучший менеджер: Петрова Анна Дмитриевна
• Лучший месяц: 2023-10

СОЗДАННЫЕ ФАЙЛЫ:
1. Отчет_по_пролонгациям_2023_с_графиками.xlsx - Полный отчет с графиками
2. Отчет_по_пролонгациям_упрощенный.xlsx - Упрощенная версия
